### GridsearchCV
Performs hyperparameteroptimisation even on estimators that don't have a *CV version

In [51]:
import pandas as pd
from sklearn.datasets import load_wine
import matplotlib.pyplot as plt
import numpy as np

wine = load_wine()
wine.keys()


dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names'])

In [52]:
df = pd.DataFrame(wine.data, columns = wine.feature_names)
df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


In [53]:
wine.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2])

## Split model into training and test data

In [54]:
from sklearn.model_selection import train_test_split

X, y = wine.data, wine.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


### Pipeline
Set up an Pipeline for execution inside gridsearchCV object.

In [55]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

scaler = StandardScaler()

pipe_knn = Pipeline([{"scaler", scaler}, ("knn", KNeighborsClassifier())])

pipe_log = make_pipeline(scaler, LogisticRegression(penalty='elasticnet', solver='saga', max_iter=10000))

pipe_forest = Pipeline([{"forest", RandomForestClassifier()}])

params_knn = {'knn__n_neighbors': list(range(1, 50))}
params_log = {'logisticregression__l1_ratio': np.linspace(0, 1, 20)}
params_forest = {'forest__n_estimators': list(range(50, 200, 10))}

classifier_knn = GridSearchCV(estimator=pipe_knn, param_grid=params_knn, cv=5, scoring="accuracy")
classifier_log = GridSearchCV(estimator=pipe_log, param_grid=params_log, cv=5, scoring="accuracy")
classifier_forest = GridSearchCV(estimator=pipe_forest, param_grid=params_forest, cv=5, scoring='accuracy')

Train the classiefiers on the training data.

In [57]:
classifier_knn.fit(X_train, y_train)
classifier_log.fit(X_train, y_train)
classifier_forest.fit(X_train, y_train)

c:\Users\Rober\.virtualenvs\MachineLearning-Joakim-Roberg-dcQuBXIe\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
119 fits failed out of a total of 245.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
119 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Rober\.virtualenvs\MachineLearning-Joakim-Roberg-dcQuBXIe\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Rober\.virtualenvs\MachineLearning-Joakim-Roberg-dcQuBXIe\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

TypeError: keywords must be strings